# HEC-RAS Steady State Flow Analysis

This notebook demonstrates how to extract and analyze steady state flow results from HEC-RAS using the ras-commander library. It showcases the new steady state functionality in `HdfResultsPlan`.

## New Steady State Methods

The library now includes full support for steady state analysis:
- `is_steady_plan()` - Check if HDF contains steady state results
- `get_steady_profile_names()` - Extract steady state profile names
- `get_steady_wse()` - Extract water surface elevations for profiles
- `get_steady_info()` - Extract steady flow metadata and attributes

## Package Installation and Environment Setup

In [ ]:
# Install ras-commander from pip (uncomment to install if needed)
# !pip install --upgrade ras-commander

# Set to False to disable plot generation for llm-friendly outputs
generate_plots = True

In [ ]:
# Import required modules
from ras_commander import *

import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# Set pandas display options
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)

## Load Bald Eagle Creek Example Project

This project contains both unsteady (Plan 01) and **steady state** (Plan 02) flow analyses.

In [ ]:
# Extract and initialize the Bald Eagle Creek project
current_dir = Path.cwd()
bald_eagle_path = current_dir / "example_projects" / "Balde Eagle Creek"

# Extract project if needed
if not bald_eagle_path.exists():
    RasExamples.extract_project("Balde Eagle Creek")
    print("Extracted Bald Eagle Creek project")
else:
    print("Bald Eagle Creek project already exists")

# Initialize the project
init_ras_project(bald_eagle_path, "6.6")
print(f"Initialized project: {ras.project_name}")

In [ ]:
# View all plans in the project
print("Plans in this project:")
ras.plan_df[['plan_number', 'Plan Title', 'unsteady_number', 'Program Version']]

## Run Steady State Plan (Plan 02)

Execute the steady state plan if results don't already exist.

In [ ]:
# Define plan number
plan_number = "02"

# Check if results exist
plan02_hdf = bald_eagle_path / "BaldEagle.p02.hdf"

if not plan02_hdf.exists():
    print(f"Running Plan {plan_number} (Steady State)...")
    success = RasCmdr.compute_plan(plan_number)
    if success:
        print(f"Plan {plan_number} executed successfully")
    else:
        print(f"Plan {plan_number} execution failed")
else:
    print(f"Plan {plan_number} results already exist")

## 1. Check if Plan Contains Steady State Results

Use `is_steady_plan()` to verify the HDF contains steady state results.

In [ ]:
# Check if this is a steady state plan
is_steady = HdfResultsPlan.is_steady_plan(plan_number)
print(f"Is Plan {plan_number} a steady state plan? {is_steady}")

## 2. Extract Steady State Profile Names

Get the list of all steady state profiles (e.g., different return periods).

In [ ]:
# Get profile names
profiles = HdfResultsPlan.get_steady_profile_names(plan_number)

print(f"Found {len(profiles)} steady state profiles:")
for i, profile in enumerate(profiles, 1):
    print(f"  {i}. {profile}")

## 3. Extract Water Surface Elevations (WSE)

Extract WSE data for specific profiles or all profiles at once.

### 3a. Extract Single Profile by Name

In [ ]:
# Extract WSE for 100-year profile
wse_100yr = HdfResultsPlan.get_steady_wse(plan_number, profile_name='100 year')

print(f"WSE Data for 100-year profile:")
print(f"Shape: {wse_100yr.shape}")
print(f"Columns: {list(wse_100yr.columns)}")
print("\nFirst 5 cross sections:")
wse_100yr.head()

### 3b. Extract Single Profile by Index

In [ ]:
# Extract WSE for first profile (0.5-year) using index
wse_05yr = HdfResultsPlan.get_steady_wse(plan_number, profile_index=0)

print(f"WSE Data for {profiles[0]} profile:")
print(f"Shape: {wse_05yr.shape}")
print("\nSummary statistics:")
wse_05yr['WSE'].describe()

### 3c. Extract All Profiles

In [ ]:
# Extract WSE for all profiles
wse_all = HdfResultsPlan.get_steady_wse(plan_number)

print(f"WSE Data for all {len(profiles)} profiles:")
print(f"Shape: {wse_all.shape}")
print(f"Columns: {list(wse_all.columns)}")
print(f"\nProfiles included: {wse_all['Profile'].unique().tolist()}")
print("\nSample data:")
wse_all.head(10)

## 4. Extract Steady Flow Metadata

Get plan information, program version, solution status, and flow file details.

In [ ]:
# Get steady flow information
steady_info = HdfResultsPlan.get_steady_info(plan_number)

print(f"Steady Flow Information ({len(steady_info.columns)} attributes):")
print("\nKey attributes:")
for col in ['Program Version', 'Solution', 'Flow Title', 'Flow Filename']:
    if col in steady_info.columns:
        print(f"  {col}: {steady_info[col].values[0]}")

print("\nAll attributes:")
steady_info.T

## 5. Visualize Water Surface Profiles

Plot WSE vs. station for different return periods.

In [ ]:
if generate_plots:
    # Create a plot comparing all profiles
    fig, ax = plt.subplots(figsize=(15, 8))
    
    # Plot each profile
    for profile in profiles:
        profile_data = wse_all[wse_all['Profile'] == profile]
        # Convert station to numeric for plotting
        stations = pd.to_numeric(profile_data['Station'], errors='coerce')
        ax.plot(stations, profile_data['WSE'], label=profile, linewidth=2)
    
    ax.set_xlabel('River Station (ft)', fontsize=12)
    ax.set_ylabel('Water Surface Elevation (ft)', fontsize=12)
    ax.set_title('Steady State Water Surface Profiles\nBald Eagle Creek', fontsize=14)
    ax.legend(title='Return Period', loc='best', fontsize=10)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print profile comparison stats
    print("\nProfile Comparison (Maximum WSE):")
    for profile in profiles:
        max_wse = wse_all[wse_all['Profile'] == profile]['WSE'].max()
        print(f"  {profile:10s}: {max_wse:.2f} ft")

## 6. Analyze WSE Differences Between Profiles

Compare water surface elevations between different return periods.

In [ ]:
# Create a pivot table for easy comparison
wse_pivot = wse_all.pivot_table(
    index=['River', 'Reach', 'Station'],
    columns='Profile',
    values='WSE'
)

print("Water Surface Elevations by Profile and Station:")
print("\nFirst 10 stations:")
wse_pivot.head(10)

In [ ]:
# Calculate differences between profiles
if '100 year' in wse_pivot.columns and '.5 year' in wse_pivot.columns:
    wse_pivot['Diff_100yr_vs_05yr'] = wse_pivot['100 year'] - wse_pivot['.5 year']
    
    print("\nDifference between 100-year and 0.5-year profiles:")
    print(f"  Maximum difference: {wse_pivot['Diff_100yr_vs_05yr'].max():.2f} ft")
    print(f"  Minimum difference: {wse_pivot['Diff_100yr_vs_05yr'].min():.2f} ft")
    print(f"  Average difference: {wse_pivot['Diff_100yr_vs_05yr'].mean():.2f} ft")
    
    print("\nStations with largest differences:")
    top_diff = wse_pivot.nlargest(5, 'Diff_100yr_vs_05yr')[['100 year', '.5 year', 'Diff_100yr_vs_05yr']]
    print(top_diff)

## 7. Plot WSE Differences

In [ ]:
if generate_plots and 'Diff_100yr_vs_05yr' in wse_pivot.columns:
    fig, ax = plt.subplots(figsize=(15, 6))
    
    # Get station values
    stations = pd.to_numeric(wse_pivot.index.get_level_values('Station'), errors='coerce')
    
    # Plot differences
    ax.plot(stations, wse_pivot['Diff_100yr_vs_05yr'], 'b-', linewidth=2)
    ax.fill_between(stations, 0, wse_pivot['Diff_100yr_vs_05yr'], alpha=0.3)
    
    ax.set_xlabel('River Station (ft)', fontsize=12)
    ax.set_ylabel('WSE Difference (ft)', fontsize=12)
    ax.set_title('Water Surface Elevation Difference\n100-year vs 0.5-year Profiles', fontsize=14)
    ax.grid(True, alpha=0.3)
    ax.axhline(y=0, color='k', linestyle='--', linewidth=0.5)
    
    plt.tight_layout()
    plt.show()

## Summary

This notebook demonstrated the new steady state functionality in ras-commander:

1. ✅ Checked if a plan contains steady state results
2. ✅ Extracted profile names for different return periods
3. ✅ Retrieved WSE data for individual and all profiles
4. ✅ Accessed steady flow metadata and attributes
5. ✅ Visualized water surface profiles
6. ✅ Analyzed differences between profiles

These tools enable comprehensive steady state flow analysis and comparison of hydraulic conditions across different design storms or flow scenarios.